In [1]:
import json
import os
import numpy as npy

import pickle
import random 
from random import sample

plot_type = 'bar'
data_dir_name = 'ct_'+plot_type


In [2]:

with open(f'plots/s_{plot_type}_annotation.pkl', 'rb') as handle:
    raw_s_pickle = pickle.load(handle)

with open(f'plots/h_{plot_type}_annotation.pkl', 'rb') as handle:
    raw_h_pickle = pickle.load(handle)
    
with open(f'plots/h11_{plot_type}_annotation.pkl', 'rb') as handle:
    raw_h_pickle += pickle.load(handle)

t2p = 0
with open(f'plots/{plot_type}_annotation.pkl', 'rb') as handle:
    raw_t2p_pickle = pickle.load(handle)
    t2p+=len(raw_t2p_pickle)
 #table 2 plot




In [3]:
# raw_s_pickle[0].keys(), raw_s_pickle[0]['bboxes']

In [4]:
from tqdm import tqdm 
import cv2

#remove overlapped bbox
#xml bbox --> yolo bbox

def intersects(bbox_a, bbox_b):
    return not (bbox_a[2] <= bbox_b[0] or bbox_a[0] >= bbox_b[2] or bbox_a[3] <= bbox_b[1] or bbox_a[1] >= bbox_b[3])

def check_overlapped_text_bbox(bboxes):
    text_bboxes = []

    for bbox in bboxes:
        label, raw_bbox, gt_text = bbox
        if gt_text:
            text_bboxes.append(raw_bbox)
    for i, text_bbox_a in enumerate(text_bboxes):
        for j, text_bbox_b in enumerate(text_bboxes[i+1:]):
            if intersects(text_bbox_a, text_bbox_b):
                return True

def generate_bbox(image_dir, coordinates, img_shape, margin=0):
    #xml(xmin, ymin, xmax, ymax) -> yolo(xcenter, ycenter, width, height) 
    xmin, ymin, xmax, ymax = coordinates
    if image_dir not in ["human", "sample"]:
        y_max_invert = img_shape[0] - ymin
        y_min_invert = img_shape[0] - ymax
        ymin = y_min_invert
        ymax = y_max_invert

    width = abs(xmin - xmax)
    height = abs(ymin - ymax)
    x_center = (xmin + width / 2) / img_shape[1]
    y_center = (ymin + height / 2) / img_shape[0]

    width = width / img_shape[1]
    height = height / img_shape[0]

    yolo_bbox = [x_center, y_center, width, height]



    overval = False
    for i in yolo_bbox:
        if i < 0 or i > 1: overval = True
    
    return yolo_bbox, overval

def raw_to_data(anno_list, image_dir):
    new_pickle = []
    for anno_id, anno in enumerate(tqdm(anno_list, leave=True, position=0)):
        image_name = anno['image']
        overlapped = check_overlapped_text_bbox(anno['bboxes'])
        if overlapped:
            # print(f"overlapped text bbox in {image_dir}/{image_name}")
            continue
        img_data = cv2.imread(f'./plots/{image_dir}/{image_name}')
        new_bboxes = []
        for bbox in anno['bboxes']:
            label, raw_bbox, gt_text = bbox
            yolo_bbox, overval = generate_bbox(image_dir, raw_bbox, img_data.shape[:-1])
            if overval:
                # print(f"bbox not in range 0~1 {image_dir}/{image_name}")
                continue
            else: 
                new_bboxes.append((label, yolo_bbox, gt_text))
        anno['bboxes'] = new_bboxes
        new_pickle.append(anno)
    return new_pickle


In [5]:
    
s_pickle = raw_to_data(raw_s_pickle, "sample")
h_pickle = raw_to_data(raw_h_pickle, "human")
t2p_pickle = raw_to_data(raw_t2p_pickle, plot_type)



100%|██████████| 1665/1665 [04:12<00:00,  6.58it/s]


In [6]:

print('#-----before  pickle-----#')
print(f"s:",len(raw_s_pickle))
print(f"h:", len(raw_h_pickle))
print(f"t2p:",t2p)
print('#-----after pickle-----#')
print(f"s {plot_type}:",len(s_pickle))
print(f"h:", len(h_pickle))
print(f"t2p:",len(t2p_pickle))
# #-----before  pickle-----#
# s: 3
# h: 105
# t2p: 1665
# #-----after pickle-----#
# s: 3
# h: 95
# t2p: 1565

#-----before  pickle-----#
s: 3
h: 105
t2p: 1665
#-----after pickle-----#
s bar: 3
h: 95
t2p: 1565


In [7]:
a_file_names = [ann['image'] for ann in s_pickle]
b_file_names = [ann['image'] for ann in h_pickle]
c_file_names = [ann['image'] for ann in t2p_pickle]

a_test_num = len(a_file_names)
b_test_num = 43
c_test_num = int(len(c_file_names)*0.05)

np.random.seed(123) # random한 것은 반드시!!!! seed를 위에 붙여줘야 고정됨.건드리지 말것.
random.seed(123) # random한 것은 반드시!!!! seed를 위에 붙여줘야 고정됨.건드리지 말것.
np.random.shuffle(a_file_names)
np.random.shuffle(b_file_names)
np.random.shuffle(c_file_names)

np.random.seed(123) # random한 것은 반드시!!!! seed를 위에 붙여줘야 고정됨.건드리지 말것.
random.seed(123) # random한 것은 반드시!!!! seed를 위에 붙여줘야 고정됨.건드리지 말것.
#sample
a_test_names_set = set(list(sample(a_file_names, a_test_num)))
#human
b_test_names_set = set(list(sample(b_file_names, b_test_num)))
#automatic
c_test_names_set = set(list(sample(c_file_names, c_test_num)))

trnval_file_names = []
for i in a_file_names:
    if i in a_test_names_set: continue
    else:
        trnval_file_names.append(i)
for i in b_file_names:
    if i in b_test_names_set: continue
    else:
        trnval_file_names.append(i)
for i in c_file_names:
    if i in c_test_names_set: continue
    else: 
        trnval_file_names.append(i)

np.random.seed(123) # random한 것은 반드시!!!! seed를 위에 붙여줘야 고정됨. 건드리지 말것.
random.seed(123) # random한 것은 반드시!!!! seed를 위에 붙여줘야 고정됨. 건드리지 말것.
trn_names_set = sample(trnval_file_names, int(len(trnval_file_names)*0.95))
train_num = len(trn_names_set)
val_names_set = [i for i in trnval_file_names if i not in trn_names_set]
valid_num = len(val_names_set)

print("train:", len(trn_names_set))
print("valid:", len(val_names_set))
print("a test:", len(a_test_names_set))
print("b test:", len(b_test_names_set))
print("c test:", len(c_test_names_set))
print("total:", len(trn_names_set)+len(val_names_set)+len(a_test_names_set)+len(b_test_names_set)+len(c_test_names_set) )


train: 1462
valid: 77
a test: 3
b test: 43
c test: 78
total: 1663


In [8]:
####################################
#GUIDE LINE
###################################
"""
필요한 코드 
-labels 폴더 내 train / valid / testA / testB / test C 폴더 내 {이미지 이름}.txt 파일 생성
-images 폴더 내 train / valid / testA / testB / test C 복사된 이미지 파일 
"""

'\n필요한 코드 \n-labels 폴더 내 train / valid / testA / testB / test C 폴더 내 {이미지 이름}.txt 파일 생성\n-images 폴더 내 train / valid / testA / testB / test C 복사된 이미지 파일 \n'

In [9]:
idx2label ={
  0: "title",
  1: "t_legend",
  2: "x_label",
  3: "y_label",
  4: "x_tick",
  5: "y_tick",
  6: "val",
  7: "pietick_0",
  8: "pietick_1",
  9: "pietick_2",
  10: "pietick_3",
  11: "pietick_4",
  12: "pietick_5",
  13: "pietick_6",
  14: "pietick_7",
  15: "pietick_8",
  16: "pietick_9",
  17: "pietick_10",
  18: "pietick_11",
  19: "pietick_12",
  20: "pietick_13",
  21: "pietick_14",
  22: "pieval_0",
  23: "pieval_1",
  24: "pieval_2",
  25: "pieval_3",
  26: "pieval_4",
  27: "pieval_5",
  28: "pieval_6",
  29: "pieval_7",
  30: "pieval_8",
  31: "pieval_9",
  32: "pieval_10",
  33: "pieval_11",
  34: "pieval_12",
  35: "pieval_13",
  36: "pieval_14",
  37: "pieratio_0",
  38: "pieratio_1",
  39: "pieratio_2",
  40: "pieratio_3",
  41: "pieratio_4",
  42: "pieratio_5",
  43: "pieratio_6",
  44: "pieratio_7",
  45: "pieratio_8",
  46: "pieratio_9",
  47: "pieratio_10",
  48: "pieratio_11",
  49: "pieratio_12",
  50: "pieratio_13",
  51: "pieratio_14",
  52: "v_legend",
  53: "line_0",
  54: "line_1",
  55: "line_2",
  56: "line_3",
  57: "line_4",
  58: "line_5",
  59: "line_6",
  60: "line_7",
  61: "line_8",
  62: "line_9",
  63: "bar",
  64: "wedge_0",
  65: "wedge_1",
  66: "wedge_2",
  67: "wedge_3",
  68: "wedge_4",
  69: "wedge_5",
  70: "wedge_6",
  71: "wedge_7",
  72: "wedge_8",
  73: "wedge_9",
  74: "wedge_10",
  75: "wedge_11",
  76: "wedge_12",
  77: "wedge_13",
  78: "wedge_14",
}

idx2label_new ={
  0: "title",
  1: "t_legend",
  2: "x_label",
  3: "y_label",
  4: "x_tick",
  5: "y_tick",
  6: "val",
  7: "v_legend",
  8: "line",
  9: "bar",
  10: "wedge",
}

label2idx = dict()
for k, v in idx2label_new.items():
  label2idx[v] = k

def trans_cat(cat_text):
  if cat_text.startswith('pie'): cat_text = 'val'
  elif cat_text.startswith('wedge'): cat_text = 'wedge'
  elif cat_text.startswith('line'): cat_text = 'line'
  return label2idx[cat_text] 

In [10]:
import os
from shutil import copyfile

def mkdirs(newdir,mode=777):
    try:
        os.makedirs(newdir, mode)
    except OSError as err:
        return err

def set_img_label(anno_list, anno_name_list, split_list, split_name, dataset_name): 
    # [s_pickle], ["sample"], a_test_names_set, "testA", "abc_bar"
    image_path = f'{dataset_name}/images/{split_name}'
    label_path = f'{dataset_name}/labels/{split_name}'
    mkdirs(image_path)
    mkdirs(label_path)

    if split_name in ["train", "valid"] and dataset_name.startswith("ctv_", "cv_", "ct_"):
        anno_list.pop(0)
        anno_name_list.pop(0)
    cnt = 0 
    for annos, names in zip(tqdm(anno_list, ncols=100, leave=True, position=0), anno_name_list):
        image_source = f'plots/{names}'
        for anno_id, anno in enumerate(annos):
            image_name = anno['image']
            if image_name in split_list:
                cnt += 1
                os.system('chmod 777 -R *')
                copyfile(os.path.join(image_source, image_name) , os.path.join(image_path, image_name))
                f = open(os.path.join(label_path, image_name[:-4]+".txt"), 'w')
                for bbox in anno['bboxes']:
                    label, yolo_bbox, gt_text = bbox
                    cat = trans_cat(label)
                    if cat in np.arange(0, 7) and 't' not in dataset_name: continue#text이면서 클래스 내에 포함하면 안되는 경우: 제외
                    if cat in np.arange(7, 11) and 'v' not in dataset_name: continue#visual이면서 클래스 내에 포함하면 안되는 경우: 제외
                    xcenter, ycenter, width, height = yolo_bbox
                    row = f"{cat} {xcenter} {ycenter} {width} {height}\n"
                    f.write(row)
    print(f"total ={cnt}" )
    return 

In [11]:
# dataset= [trn_names_set, val_names_set, a_test_names_set, b_test_names_set, c_test_names_set ]
# pickle = [s_pickle, h_pickle, t2p_pickle]

# data_dir_name = 'c_'+plot_type

#< [s_pickle], "sample", a_test_names_set, "testA", data_dir_name >
set_img_label([h_pickle, t2p_pickle], ["human", plot_type], trn_names_set, "train", data_dir_name) 
set_img_label([h_pickle, t2p_pickle], ["human", plot_type], val_names_set, "valid", data_dir_name)
set_img_label([s_pickle], ["sample"], a_test_names_set, "testA", data_dir_name)
set_img_label([h_pickle], ["human"], b_test_names_set, "testB", data_dir_name)
set_img_label( [t2p_pickle], [plot_type], c_test_names_set, "testC", data_dir_name)


  0%|                                                                         | 0/1 [00:00<?, ?it/s]chmod: changing permissions of 'cv_all/labels/testA/barA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/lineA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/pieA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/barB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/lineB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/pieB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/barC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/lineC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/pieC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/train.cache': Operation not permitted
chmod: chang

total =1414


  0%|                                                                         | 0/1 [00:00<?, ?it/s]chmod: changing permissions of 'cv_all/labels/testA/barA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/lineA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/pieA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/barB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/lineB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/pieB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/barC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/lineC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/pieC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/train.cache': Operation not permitted
chmod: chang

total =73


  0%|                                                                         | 0/1 [00:00<?, ?it/s]chmod: changing permissions of 'cv_all/labels/testA/barA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/lineA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/pieA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/barB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/lineB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/pieB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/barC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/lineC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/pieC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/train.cache': Operation not permitted
chmod: chang

total =3


  0%|                                                                         | 0/1 [00:00<?, ?it/s]chmod: changing permissions of 'cv_all/labels/testA/barA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/lineA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/pieA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/barB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/lineB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/pieB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/barC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/lineC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/pieC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/train.cache': Operation not permitted
chmod: chang

total =43


  0%|                                                                         | 0/1 [00:00<?, ?it/s]chmod: changing permissions of 'cv_all/labels/testA/barA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/lineA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testA/pieA.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/barB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/lineB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testB/pieB.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/barC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/lineC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/testC/pieC.cache': Operation not permitted
chmod: changing permissions of 'cv_all/labels/train.cache': Operation not permitted
chmod: chang

total =78
